In [2]:
import pandas as pd
from pandasql import sqldf
import datetime
from datetime import timedelta
import copy
import numpy as np

In [3]:
'''
FUNCTIONS:

import_tests
convert_to_secs
split_by_interval
group_by_BSS
join_tables
join_master
join_cos
'''

'\nFUNCTIONS:\n\nimport_tests\nconvert_to_secs\nsplit_by_interval\ngroup_by_BSS\njoin_tables\njoin_master\njoin_cos\n'

In [4]:
def import_tests(labels, testnum):
    '''Imports CSV files (for our purposes, 2 files) into pandas DFs
    
    Parameters
    ----------
    labels : list of 2 strings
        Intials/labels of desired tests, ex: ['CF', 'CB']
    testnum : int
        test number according to labeling      
    
    '''
    output=[]
    
    #iterates through each user
    for i in range(len(labels)):
        j=labels[i]
        #reads corresponding CSV as a Pandas DataFrame
        exec("output.append(pd.read_csv('../../CSV_Files/test_%s_%s.csv'))" %(testnum, j))
    return output

In [5]:
#TODO use datetime objects, use df.apply
def convert_to_secs(list_of_tables):
    '''Adds a column <Seconds> to both DFs and converts Time to Seconds
    
    Parameters
    ----------
    list_of_tables : list of DFs
        list of DFs containing Wi-Fi scans
    
    Returns
    ----------
    list_of_tables : list of DFs
        list of DFs containing Wi-Fi scans
    '''
    
    #iterates through DataFrames
    for i in list_of_tables:
        
        #creates a column called Seconds
        i['Seconds']=0
        
        #iterates through row (df.apply would be much more efficient)
        for j in range(0, len(i)):
            total_secs=0
            
            #interpretates time and calculates total seconds
            if 'PM' in i['Time'][j]:
                time_parsed=i['Time'][j].strip(' PM').split(':')
                tup=(int(time_parsed[0]), int(time_parsed[1]), int(time_parsed[2]))
                total_secs=(tup[0]+12)*60*60+tup[1]*60+tup[2]
            elif 'AM' in i['Time'][j]:
                time_parsed=i['Time'][j].strip(' PM').split(':')   
                tup=(int(time_parsed[0]), int(time_parsed[1]), int(time_parsed[2]))
                total_secs=tup[0]*60*60+tup[1]*60+tup[2]
            else:
                time_parsed=i['Time'][j].split(':')
                tup=(int(time_parsed[0]), int(time_parsed[1]), int(time_parsed[2]))
                total_secs=tup[0]*60*60+tup[1]*60+tup[2]
                
            #sets Seconds to total number of seconds. This will be used to group tests into comparison intervals
            i.at[j, 'Seconds']=total_secs
            
    return list_of_tables

In [6]:
def split_by_interval(list_of_tables, interval):
    '''Splits both DFs into multiple DFs based on a time comparison interval: For instance, if we compare 2 5-minute tests 
       and set the chunking interval in this method to 60 seconds, we would expect that the 2 dataframes will become an output  
       of 10 total dataframes, since each overall table will be divided up into 5 smaller ones. 
    
    Parameters
    ----------
    list_of_tables : list of DFs
        list of DFs containing Wi-Fi Scans
    interval: int
        chosen time comparison interval in seconds 
        
    Returns
    ----------
    output : list of DFs
        list of DFs containing Wi-Fi Scans
    '''
    output=[]
    
    #This finds the overall starting time between both tests. For example, if one user's scan began recording at 8:00:00 
    #and the other user's started at 8:00:04, this method finds the overall starting time for both tests, being 8:00:04 in this 
    #case, since we want to only look at overlapping scans between the two users. 
    
    starting_seconds = [min(i['Seconds']) for i in list_of_tables]
    start=max(starting_seconds)
    
    #Similar to the previous assignment, this will find the overall ending time between the two tests. For instance, if
    #the last scan on user 1's phone was recorded at 9:00:00 and user 2's phone picked up a final scan at 9:00:05, we determine
    #the overall ending time to be the earlier time, 9:00:00 in this case. 
    
    ending_seconds = [max(i['Seconds']) for i in list_of_tables]
    end=min(ending_seconds)
    
    #This looks through both data frames and beginning from the determined starting time, it will increment by the desired 
    #chunking interval to group the data into smaller dataframes between designated time intervals. This will follow until the 
    #determined ending time is reached.
    
    iterations=int((end-start)/interval)+1
    for i in list_of_tables:
        minimum=start
        maximum=minimum+interval
        for k in range(0, iterations):
            temp = sqldf("SELECT * FROM i WHERE Seconds >= %s AND Seconds < %s" %(minimum, maximum))
            temp['Second_Index']=minimum
            output.append(temp)
            minimum+=interval #updadate interval
            maximum+=interval
    return output

In [7]:
def group_by_BSS(list_of_tables):
    '''Groups the tables by BSS and Averages the RSSI values across these matching BSS's. If following the overall joining 
        method as defined below, the group_by_BSS method will be used on the dataframes that have already been separated into 
        designated time chunks. There will be an output with one BSS per row.
    
    Parameters
    ----------
    list_of_tables : list of DFs
        list of DFs containing Wi-Fi Scans
        
    Returns
    ----------
    output : list of DFs
        list of DFs containing Wi-Fi Scans
    '''
    output=[]
    for i in list_of_tables:
        temp=None
        dict_temp=None
        temp_std=None
        temp_std_2=None
        temp2=None
        
        dict_temp={}
        temp = i
        
        #Line below takes standard deviation of all columns in the dataframe, so that we can 
        #eventually access the std of RSSI w.r.t. BSS groupings.
        temp_std=i.groupby('BSS').std(ddof=1).reset_index()
        temp_std_2=temp_std
        for j in range(0, len(temp_std_2)):
            dict_temp[temp_std_2.at[j, 'BSS']]=temp_std_2.at[j, 'RSSI']
        
        #Averages RSSI across the specific time chunks per each BSS.
        temp2 = sqldf("SELECT SSID, BSS, AVG(RSSI) AS RSSI, COUNT(BSS) as BSS_Count, Channel, Time, Seconds, Second_Index, Date, Test, Phone FROM temp GROUP BY BSS")
        temp2['Std_dev']=0.0 #New column, sets the standard deviations in df as decimal/float values, starting at 0.0 initially for all.
        
        for j in range(0, len(temp2)):
            
            #Acesses the calculated standard deviation across the BSS's in the dictionary and resets from 0.0 above.
            temp2.at[j, 'Std_dev']=dict_temp.get(temp2.at[j, 'BSS'])
        temp2['Count'] = len(temp2)
        output.append(temp2)
        
    return output

In [8]:
def join_tables(list_of_tables):
    '''Joins two tables from the same time interval. This joins data together from the two different users.
    
    Parameters
    ----------
    list_of_tables : list of DFs
        list of DFs containing Wi-Fi Scans
        
    Returns
    ----------
    list
        a list of DFs, each that contain two tests being compared
    '''
    output_temp=[]
    for i in range(0, len(list_of_tables)):
        
        #Compares one table to all other tables in the list.
        for j in range(i+1, len(list_of_tables)):
            temp1=list_of_tables[i]
            temp2=list_of_tables[j]
            
            #Joins together if the important info is the same and the users are different. 
            output_temp.append(sqldf("SELECT A.SSID AS SSID_A, A.RSSI AS RSSI_A, A.BSS AS BSS_A, A.Time AS Time_A, A.Seconds AS Seconds_A, A.Date AS Date_A, A.Phone \
            AS Phone_A, A.Count AS Count_A, A.Std_dev AS Std_dev_A, B.SSID AS SSID_B, B.RSSI AS RSSI_B , B.BSS AS BSS_B, B.Time AS Time_B, B.Seconds AS Seconds_B, B.Date AS Date_B, B.Phone AS Phone_B, B.Count AS Count_B, \
            B.Std_dev AS Std_dev_B FROM temp1 AS A JOIN temp2 AS B ON A.SSID = B.SSID AND A.BSS = B.BSS AND \
            A.Channel = B.Channel AND A.Date=B.Date AND A.Second_Index=B.Second_Index AND A.Phone != B.Phone"))
    output=[]
    
    for i in output_temp:
        #Filter out DFs that are only NaNs - due to only 1 BSS reading.
        if i.dropna().reset_index().empty==False:
            output.append(i.dropna().reset_index())
    return output

    

In [9]:
def join_master(initials, testnum, interval):
    '''Joins specified tests, not considering unmatched networks
    
    Parameters
    ----------
    initials : list of 2 strings
        devices to compare between
    testnum : int
        test number to compare
    interval : int
        comparison interval
        
    Returns
    ----------
    list
        a list of DFs, each that contain two tests being compared
        
    '''
    step_1=import_tests(initials, testnum)
    step_2=convert_to_secs(step_1)
    step_3=split_by_interval(step_2, interval)
    step_4=group_by_BSS(step_3)
    step_5=join_tables(step_4)
    
    return step_5

In [10]:
#This general joining method is the same as join_master w/ the exception that join_cos does NOT drop na values.
def join_master_unmatched(initials, testnum, interval):
    '''Joins specified tests, leaving RSSI values of unmatched networks as NaNs
    
    Parameters
    ----------
    initials : list of 2 strings
        devices to compare between
    testnum : int
        test number to compare
    interval : int
        comparison interval
        
    Returns
    ----------
    list
        a list of DFs, each that contain two tests being compared
        
    '''
    step_1=import_tests(initials, testnum)
    step_2=convert_to_secs(step_1)
    step_3=split_by_interval(step_2, interval)
    step_4=group_by_BSS(step_3)
    output_temp=[]
    for i in range(0, len(step_4)):
        for j in range(i+1, len(step_4)):
            temp1=step_4[i]
            temp2=step_4[j]
            if temp1['Phone'][0] != temp2['Phone'][0] and temp1['Second_Index'][0]==temp2['Second_Index'][0] and temp1['Date'][0]==temp2['Date'][0]: #IF Scan interval is too small, throws error
                temp3 = pd.merge(temp1,temp2,on=['BSS'],how='outer')
                output_temp.append(temp3)
  
    output=[]
    for i in output_temp:
        if i.empty==False:
            output.append(i)

    return output
    